In [1]:
import torch
from torch.nn import Conv2d, Linear
from torchvision.models import resnet50
import os,sys
parent_dir = os.path.abspath('../')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
resnet = resnet50()

In [ ]:
resnet.conv1 = Conv2d(8, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet.fc = Linear(in_features=2048, out_features=1024, bias=True)
print(resnet)

In [ ]:
from util.frame_diff import Sandevistan
from util.dark_enhance import DarkEnhance
from torchvision.io import read_video
frames, _, info  = read_video('./Run_1_1.mp4')
print('frames',frames.shape)

In [ ]:
enhancer = DarkEnhance()
frame_diff = Sandevistan(n_frames=10,thres=1.0)
frames = enhancer(frames).transpose(3,1).unsqueeze(0)
out = frame_diff(frames.clone())
print(out.shape)

In [2]:
from model.network import SandevistanCLIP, build_model
state_dict = torch.load('/mnt/e/vit-b-16-8f.pt')['model_state_dict']
#clip = build_model(state_dict,pretrain=True)


In [3]:
print(state_dict.keys())

odict_keys(['positional_embedding', 'text_projection', 'logit_scale', 'visual.class_embedding', 'visual.positional_embedding', 'visual.proj', 'visual.conv1.weight', 'visual.ln_pre.weight', 'visual.ln_pre.bias', 'visual.transformer.resblocks.0.attn.in_proj_weight', 'visual.transformer.resblocks.0.attn.in_proj_bias', 'visual.transformer.resblocks.0.attn.out_proj.weight', 'visual.transformer.resblocks.0.attn.out_proj.bias', 'visual.transformer.resblocks.0.ln_1.weight', 'visual.transformer.resblocks.0.ln_1.bias', 'visual.transformer.resblocks.0.mlp.c_fc.weight', 'visual.transformer.resblocks.0.mlp.c_fc.bias', 'visual.transformer.resblocks.0.mlp.c_proj.weight', 'visual.transformer.resblocks.0.mlp.c_proj.bias', 'visual.transformer.resblocks.0.ln_2.weight', 'visual.transformer.resblocks.0.ln_2.bias', 'visual.transformer.resblocks.1.attn.in_proj_weight', 'visual.transformer.resblocks.1.attn.in_proj_bias', 'visual.transformer.resblocks.1.attn.out_proj.weight', 'visual.transformer.resblocks.1.at

In [ ]:
optim_dict = torch.load('/mnt/e/vit-b-16-8f.pt')['optimizer_state_dict']

In [ ]:
print(state_dict['fusion_model_state_dict'].keys())

In [ ]:
state_dict["text_projection"].shape[1]

In [ ]:
fusion_state_dict = torch.load('/mnt/e/vit-b-16-8f.pt')['fusion_model_state_dict']

In [ ]:
new_fusion_state_dict={}
for k,v in fusion_state_dict.items():
    k = k.strip('module.')
    if 'transformer' in k:
        k = k.replace('transformer','temporal')
    new_fusion_state_dict[k] = v

In [ ]:
print(new_fusion_state_dict.keys())

In [5]:
motion = {}
for k,v in state_dict.items():
    if 'visual.' in k:
        if k == 'visual.conv1.weight':
            continue
        k_ = k.replace('visual.','motion.')
        motion[k_] = v
state_dict.update(motion)

In [6]:
print(state_dict.keys())

odict_keys(['positional_embedding', 'text_projection', 'logit_scale', 'visual.class_embedding', 'visual.positional_embedding', 'visual.proj', 'visual.conv1.weight', 'visual.ln_pre.weight', 'visual.ln_pre.bias', 'visual.transformer.resblocks.0.attn.in_proj_weight', 'visual.transformer.resblocks.0.attn.in_proj_bias', 'visual.transformer.resblocks.0.attn.out_proj.weight', 'visual.transformer.resblocks.0.attn.out_proj.bias', 'visual.transformer.resblocks.0.ln_1.weight', 'visual.transformer.resblocks.0.ln_1.bias', 'visual.transformer.resblocks.0.mlp.c_fc.weight', 'visual.transformer.resblocks.0.mlp.c_fc.bias', 'visual.transformer.resblocks.0.mlp.c_proj.weight', 'visual.transformer.resblocks.0.mlp.c_proj.bias', 'visual.transformer.resblocks.0.ln_2.weight', 'visual.transformer.resblocks.0.ln_2.bias', 'visual.transformer.resblocks.1.attn.in_proj_weight', 'visual.transformer.resblocks.1.attn.in_proj_bias', 'visual.transformer.resblocks.1.attn.out_proj.weight', 'visual.transformer.resblocks.1.at

In [7]:
clip = build_model(state_dict,pretrain=True, motion_layers=4)

img transformer layers: 12
loading clip pretrained model!
The following sub-module is not loaded
_IncompatibleKeys(missing_keys=['motion.conv1.weight', 'motion.transformer.resblocks.0.ln_1_y.weight', 'motion.transformer.resblocks.0.ln_1_y.bias', 'motion.transformer.resblocks.1.ln_1_y.weight', 'motion.transformer.resblocks.1.ln_1_y.bias', 'motion.transformer.resblocks.2.ln_1_y.weight', 'motion.transformer.resblocks.2.ln_1_y.bias', 'motion.transformer.resblocks.3.ln_1_y.weight', 'motion.transformer.resblocks.3.ln_1_y.bias', 'temporal.cross_resblock.attn.in_proj_weight', 'temporal.cross_resblock.attn.in_proj_bias', 'temporal.cross_resblock.attn.out_proj.weight', 'temporal.cross_resblock.attn.out_proj.bias', 'temporal.cross_resblock.ln_1.weight', 'temporal.cross_resblock.ln_1.bias', 'temporal.cross_resblock.mlp.c_fc.weight', 'temporal.cross_resblock.mlp.c_fc.bias', 'temporal.cross_resblock.mlp.c_proj.weight', 'temporal.cross_resblock.mlp.c_proj.bias', 'temporal.cross_resblock.ln_2.weight',

In [ ]:
_state_dict = clip.state_dict()

In [ ]:
clip.load_state_dict(new_fusion_state_dict, strict=False)

In [ ]:
torch.save({'model_state_dict': clip.state_dict(), 'optimizer_state_dict':optim_dict},'/mnt/e/vit-b-16-8f_modified.pt')

In [ ]:
a = torch.rand(6,8,3,224,224).half().cuda()
b = torch.randint(10,(10,77)).cuda()
out = clip.encode_image(a)

In [ ]:
print(out.shape)

In [ ]:
out[0].sum().backward()

In [ ]:
a = torch.zeros(12,12)
b = torch.tensor([[7, 1, 9, 0, 0, 0, 3, 0, 9, 7, 2, 7]],dtype=torch.long)
u,b = torch.unique(b,return_inverse=True,sorted=False)
#u,b = torch.unique(b,return_inverse=True,sorted=False)
print(b)
c = torch.ones(1,12)
a = a.scatter(0,index=b,src=c)
print(a)

In [ ]:
torch.eq(b,b.T).to(torch.float16)

In [ ]:
# tensor([7, 1, 9, 0, 0, 0, 3, 0, 9, 7, 2, 7])
# tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
#         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
#         [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
#         [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
#         [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
#         [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
#         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
#         [0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.],
#         [0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
#         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
#         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
#         [1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.]], device='cuda:0',
#        dtype=torch.float16)

In [ ]:
a = torch.rand(6,8,3,224,224).half().cuda()
out = clip.encode_image(a)
print(out.shape)

In [ ]:
b = torch.randint(10,(160,77)).cuda()
out = clip.encode_text(b)
print(out.dtype)

In [ ]:
print(sys.getsizeof(out.storage()))

In [ ]:
a = torch.tensor([[0,6,3,2,5],[7,4,5,3,0],[0,1,2,4,5]])
print(a.shape)
b = torch.tensor([0,8,2]).unsqueeze(-1)
print(b.shape)
torch.any(torch.eq(a,b),dim=-1)

In [ ]:
a = torch.tensor([0,1,2])
b = torch.tensor([0,8,2])
torch.eq(a,b)